# Oxygen Abundance

In [1]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde

In [2]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings

In [3]:
figure_setup()
settings = Settings()

In [4]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

In [5]:
SAMPLE = [f"au{i}_or_l4_s127" for i in settings.groups["Included"]]

In [6]:
def read_data(simulation: str) -> pd.DataFrame:
    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_metal_abundance(of="Fe", to='H')
    s.add_metal_abundance(of='O', to="Fe")
    s.tag_particles_by_region(
        disc_std_circ=DISC_STD_CIRC,
        disc_min_circ=DISC_MIN_CIRC,
        cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
        bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    props = {
        "ComponentTag": s.region_tag[is_real_star & is_main_obj],
        "[Fe/H]": s.metal_abundance["Fe/H"][is_real_star & is_main_obj],
        "[O/Fe]": s.metal_abundance["O/Fe"][is_real_star & is_main_obj],
        }

    return pd.DataFrame(props)

In [7]:
def get_sample_data(simulations: list) -> pd.DataFrame:

    settings = Settings()
    props = {"Simulation": simulations}

    data = np.zeros((len(simulations), 10))
    for i, simulation in enumerate(simulations):
        df = read_data(simulation)
        data[i, -2] = np.nanmedian(df["[O/Fe]"])
        data[i, -1] = np.nanmedian(df["[Fe/H]"])
        for j in range(len(settings.components)):
            data[i, j] = np.nanmedian(
                df["[O/Fe]"][df["ComponentTag"] == j])
            data[i, j + 4] = np.nanmedian(
                df["[Fe/H]"][df["ComponentTag"] == j])
    
    for j in range(len(settings.components)):
        props[f"[O/Fe]_{settings.components[j]}"] = data[:, j]
        props[f"[Fe/H]_{settings.components[j]}"] = data[:, j + 4]
    props["[O/Fe]_All"] = data[:, -2]
    props["[Fe/H]_All"] = data[:, -1]
    
    return pd.DataFrame(props)

In [10]:
file_path = "../results/sample_median_OFe_FeH_abundances_by_component.csv"
try:
    stats = pd.read_csv(file_path)
except FileNotFoundError:
    stats = get_sample_data(SAMPLE)
    stats.to_csv(file_path)

## median([O/Fe]) vs median([Fe/H]) Scatter

In [11]:
markers = list(settings.component_markers.values())
colors = list(settings.component_colors.values())
labels = list(settings.component_labels.values())

In [13]:
fig, axs = plt.subplots(
    figsize=(8.0, 2.0), nrows=1, ncols=5, sharey=True, sharex=True,
    gridspec_kw={"hspace": 0.0, "wspace": 0.0})

for ax in axs.flatten():
    ax.grid(True, ls='-', lw=0.25, c="gainsboro")
    ax.tick_params(which='both', direction="in")
    ax.set_xlim(-1.0, 0.8)
    ax.set_xticks([-0.8, -0.4, 0, 0.4])
    ax.set_ylabel('[O/Fe]')
    ax.set_ylim(0.18, 0.32)
    ax.set_yticks([0.20, 0.22, 0.24, 0.26, 0.28, 0.3])
    ax.set_xlabel('[Fe/H]')
    ax.set_axisbelow(True)
    ax.label_outer()

axs[0].scatter(
    stats["[Fe/H]_All"], stats["[O/Fe]_All"],
    c='k', zorder=10, s=20, linewidths=0.4, edgecolors="white", marker='o')
for c in settings.components:
    axs[0].scatter(
        stats[f"[Fe/H]_{c}"], stats[f"[O/Fe]_{c}"],
        c="silver", zorder=5,
        s=20, linewidths=0.4, edgecolors="white",
        marker=settings.component_markers[c])
fe_pdf = gaussian_kde(stats[f"[Fe/H]_All"].to_numpy(dtype=np.float64))
pdf_x = np.linspace(-1.0, 0.6, 100)
pdf_y = fe_pdf(pdf_x) / np.max(fe_pdf(pdf_x)) * 0.02 + ax.get_ylim()[0]
axs[0].fill_between(
    x=pdf_x, y1=-1.0, y2=pdf_y, edgecolor="black", facecolor="#00000030")
o_pdf = gaussian_kde(stats[f"[O/Fe]_All"].to_numpy(dtype=np.float64))
pdf_y = np.linspace(0.18, 0.32, 100)
pdf_x = o_pdf(pdf_y) / np.max(o_pdf(pdf_y)) * 0.3
axs[0].fill_betweenx(
    y=pdf_y, x1=-pdf_x + 0.8, x2=0.8, edgecolor="black", facecolor="#00000030")

for i in range(4):
    ax = axs.flatten()[i + 1]
    ax.scatter(
        stats["[Fe/H]_All"], stats["[O/Fe]_All"],
        c="silver", zorder=5, s=20, linewidths=0.4,
        edgecolors="white", marker='o')
    for j in range(len(settings.components)):
        label = labels[j] if j == i else None
        color = colors[j] if j == i else "silver"
        zorder = 10 if j == i else 5
        ax.scatter(
            stats[f"[Fe/H]_{settings.components[j]}"],
            stats[f"[O/Fe]_{settings.components[j]}"],
            c=color, label=label, zorder=zorder,
            s=20, linewidths=0.4, edgecolors="white", marker=markers[j])
        
        if j == i:
            fe_pdf = gaussian_kde(
                stats[f"[Fe/H]_{settings.components[j]}"].to_numpy(
                    dtype=np.float64))
            pdf_x = np.linspace(-1.0, 0.6, 100)
            pdf_y = fe_pdf(pdf_x) / np.max(fe_pdf(pdf_x)) * 0.02 \
                + ax.get_ylim()[0]
            ax.fill_between(
                x=pdf_x, y1=-1.0, y2=pdf_y, edgecolor=color,
                facecolor=color + "30")
            
            o_pdf = gaussian_kde(
                stats[f"[O/Fe]_{settings.components[j]}"].to_numpy(
                    dtype=np.float64))
            pdf_y = np.linspace(0.18, 0.32, 100)
            pdf_x = o_pdf(pdf_y) / np.max(o_pdf(pdf_y)) * 0.3
            ax.fill_betweenx(
                y=pdf_y, x1=-pdf_x + 0.8, x2=0.8, edgecolor=color,
                facecolor=color + "30")

axs[0].text(
    x=0.05, y=0.95, size=8.0, ha="left", va="top", s=r"$\textbf{All}$",
    c="black", transform=axs[0].transAxes)
for i, c in enumerate(settings.components):
    axs[i + 1].text(
        x=0.05, y=0.95, size=8.0, ha="left", va="top",
        s=r"$\textbf{" + settings.component_labels[c] + "}$",
        c=settings.component_colors[c], transform=axs[i + 1].transAxes)

fig.savefig(f"../images/metallicity/OFe_vs_FeH_included_s127{SUFFIX}.pdf")
plt.close(fig)